In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.6 MB/s eta 0:00:00


In [ ]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

import transformers
from transformers import BertTokenizer, BertModel
from transformers import RobertaTokenizer, RobertaModel

import torch
from torch import cuda
from tqdm import tqdm_notebook as tqdm
device = 'cuda' if cuda.is_available() else 'cpu'

device

'cuda'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
abusive_data=pd.read_csv('/content/gdrive/MyDrive/NLP_Project/abusive/processed_abusive_data.csv',on_bad_lines='skip')
abusive_data = shuffle(abusive_data)
abusive_data = abusive_data.dropna().reset_index(drop=True)
abusive_data= abusive_data.reset_index(drop=True)
abusive_data['toxic']=abusive_data['toxic'].astype(int)
abusive_data['sarcastic']=abusive_data['sarcastic'].astype(int)
abusive_data=abusive_data.drop('sarcastic', axis=1)
abusive_data

,text,toxic,processed_text
0,how dare you criticise the hard working and un...,1,dare criticise hard work rat mod urban bastard...
1,california does indeed get more than of its el...,0,california indeed get electrical power renewab...
2,like everyone else get it m kka,0,like everyone else get kka
3,I spy a couple thirsty hoes on insta.,1,spy couple thirsty hoe insta
4,fuck chester marcol fuck chester marcol fuc...,1,fuck chester marcol fuck chester marcol fuck c...
...,...,...,...
204495,i see your point thanks for making it clear ...,0,see point thanks make clear exception live dea...
204496,Im not tongue kissing these bitches tho,1,im tongue kiss bitch tho
204497,i dont understand why i cant add me on the lis...,0,dont understand cant add list programmer page ...
204498,let s first explore if we re still discussing...,0,let first explore still discuss story denegrat...


In [ ]:
sarcasm_data=pd.read_csv('/content/gdrive/MyDrive/NLP_Project/sarcasm/processed_sarcasm_data.csv',on_bad_lines='skip')
sarcasm_data = shuffle(sarcasm_data)
sarcasm_data = sarcasm_data.dropna().reset_index(drop=True)
sarcasm_data= sarcasm_data.reset_index(drop=True)
sarcasm_data['toxic']=sarcasm_data['toxic'].astype(int)
sarcasm_data['sarcastic']=sarcasm_data['sarcastic'].astype(int)
sarcasm_data=sarcasm_data.drop('toxic', axis=1)
sarcasm_data

,text,sarcastic,processed_text
0,I know. I'm a Maker. You forgot the,1,know maker forget
1,Police praise A-League fans for overall good b...,1,police praise league fan overall good behaviou...
2,To the tune of Agadoo? Waterloo?,0,tune agadoo waterloo
3,Not sure if you're a troll or just fucking stu...,0,sure troll fucking stupid
4,The political ignorance is strong here on Redd...,1,political ignorance strong reddit know tpp lit...
...,...,...,...
204429,"On a similar note, I'm so glad Grawp's storyli...",0,similar note glad grawp storyline significantl...
204430,Cheeky Nando's with my mates after a piss up w...,0,cheeky nando mat piss bird lads go mental
204431,carriage ways allow people to look for parking...,0,carriage way allow people look park traffic ci...
204432,"Damn, this gets reposted once each week at lea...",1,damn get reposted week least op clearly say ev...


In [ ]:
abusive_data.to_csv('/content/gdrive/MyDrive/NLP_Project/abusive/abusive_data.csv', index=False)

In [ ]:
sarcasm_data.to_csv('/content/gdrive/MyDrive/NLP_Project/sarcasm/sarcasm_data.csv', index=False)

In [ ]:
def train_test_split(df, train_percent=0.7, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent*m)
    train = df.iloc[perm[:train_end]]
    test = df.iloc[perm[train_end:]]
    return train, test

In [ ]:
abusive_train,abusive_test =train_test_split(abusive_data, seed=123)

In [ ]:
sarcasm_data_X=sarcasm_data['processed_text']
sarcasm_data_y=sarcasm_data['sarcastic']

In [ ]:
len(sarcasm_data_X)

204434

In [ ]:
abusive_train_X=abusive_train['processed_text']
abusive_train_y=abusive_train['toxic']
abusive_test_X=abusive_test['processed_text']
abusive_test_y=abusive_test['toxic']

In [ ]:
abusive_train_X=abusive_train_X.reset_index(drop=True)
abusive_train_y=abusive_train_y.reset_index(drop=True)
abusive_test_X=abusive_test_X.reset_index(drop=True)
abusive_test_y=abusive_test_y.reset_index(drop=True)

In [ ]:
sarcasm_data_X=sarcasm_data_X.reset_index(drop=True)
sarcasm_data_y=sarcasm_data_y.reset_index(drop=True)

In [ ]:
sarcasm_data_y=sarcasm_data_y.to_numpy()

In [ ]:
abusive_train_y=abusive_train_y.to_numpy()
abusive_test_y=abusive_test_y.to_numpy()

In [ ]:
class SingleLabelDataset(torch.utils.data.Dataset):

    def __init__(self, text, labels, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = text
        self.targets = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            str(text),
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.long)
        }

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self, NUM_OUT):
        super(BERTClass, self).__init__()
                   
        self.l1 = BertModel.from_pretrained("bert-base-uncased")
        self.linear = torch.nn.Linear(768, 256)
        self.classifier = torch.nn.Linear(256, NUM_OUT)
        self.dropout = torch.nn.Dropout(0.3)
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.linear(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
def loss_fn(outputs, targets):
    targets=targets.unsqueeze(1)
    with torch.autocast('cuda'):
      loss=torch.nn.BCEWithLogitsLoss()(outputs, targets.float())
    return loss

def train(model, training_loader, optimizer):
    model.train()
    for data in tqdm(training_loader):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        outputs = model(ids, mask, token_type_ids)
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss
    
def validation(model, testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for data in testing_loader:
            targets = data['targets']
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids)
            outputs = torch.sigmoid(outputs).cpu().detach()
            fin_outputs.extend(outputs)
            fin_targets.extend(targets)
    return torch.stack(fin_outputs), torch.stack(fin_targets)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
MAX_LEN = 128
BATCH_SIZE = 64
EPOCHS = 3
NUM_OUT = 1
LEARNING_RATE = 2e-05

abusive_training_data = SingleLabelDataset(abusive_train_X, torch.from_numpy(abusive_train_y), tokenizer, MAX_LEN)
abusive_test_data = SingleLabelDataset(abusive_test_X, torch.from_numpy(abusive_test_y), tokenizer, MAX_LEN)

train_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }    

abusive_training_loader = torch.utils.data.DataLoader(abusive_training_data, **train_params)
abusive_testing_loader = torch.utils.data.DataLoader(abusive_test_data, **test_params)

In [ ]:
params = {'batch_size': BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }   

In [ ]:
sarcastic_data = SingleLabelDataset(sarcasm_data_X, torch.from_numpy(sarcasm_data_y), tokenizer, MAX_LEN)
sarcastic_loader = torch.utils.data.DataLoader(sarcastic_data, **params)

In [ ]:
abusive_model = BERTClass(NUM_OUT)
abusive_model.to(device)    
optimizer = torch.optim.Adam(params = abusive_model.parameters(), lr=LEARNING_RATE)
for epoch in range(EPOCHS):
    loss = train(abusive_model, abusive_training_loader, optimizer)
    print(f'Epoch: {epoch}, Loss:  {loss.item()}')  

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-13-d724496dfb9a>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tq

  0%|          | 0/2237 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
<ipython-input-11-af2b98af4c94>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'targets': torch.tensor(self.targets[index], dtype=torch.long)


Epoch: 0, Loss:  0.1635320484638214


  0%|          | 0/2237 [00:00<?, ?it/s]

Epoch: 1, Loss:  0.043078020215034485


  0%|          | 0/2237 [00:00<?, ?it/s]

Epoch: 2, Loss:  0.08497437089681625


In [ ]:
attributes=['toxic']

In [ ]:
a_guess, a_targs = validation(abusive_model, abusive_testing_loader)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
<ipython-input-11-af2b98af4c94>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'targets': torch.tensor(self.targets[index], dtype=torch.long)


In [ ]:
a_guess

tensor([[1.1214e-01],
        [5.8440e-01],
        [9.1994e-05],
        ...,
        [9.9988e-01],
        [4.7144e-04],
        [9.4338e-05]])

In [ ]:
a_label_predictions=[]
for i in range(len(a_guess)):
  temp=[]
  if a_guess[i] > 0.7:
    temp.append(1) 
  else:
    temp.append(0)

  a_label_predictions.append(temp)

In [ ]:
print('arracy on test set {}'.format(accuracy_score(a_label_predictions, a_targs)))

arracy on test set 0.9616462917685411


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(a_label_predictions, a_targs))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97     44308
           1       0.91      0.96      0.93     17042

    accuracy                           0.96     61350
   macro avg       0.95      0.96      0.95     61350
weighted avg       0.96      0.96      0.96     61350



In [ ]:
import pickle
pickle.dump(abusive_model, open('/content/gdrive/MyDrive/NLP_Project/abusive/a_model.pkl', 'wb'))
a_pickled_model = pickle.load(open('/content/gdrive/MyDrive/NLP_Project/abusive/a_model.pkl', 'rb'))

In [ ]:
sarcastic_guess, sarcastic_targs = validation(abusive_model, sarcastic_loader)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
<ipython-input-11-af2b98af4c94>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'targets': torch.tensor(self.targets[index], dtype=torch.long)


In [ ]:
len(sarcastic_guess)

204434

In [ ]:
sarcastic_label_predictions=[]
for i in range(len(sarcastic_guess)):
  temp=[]
  if sarcastic_guess[i] > 0.7:
    temp.append(1) 
  else:
    temp.append(0)

  sarcastic_label_predictions.append(temp)

In [ ]:
len(sarcastic_label_predictions)

204434

In [ ]:
flat_list = []
for sublist in sarcastic_label_predictions:
    for item in sublist:
        flat_list.append(item)

In [ ]:
sarcasm_data['predictive_toxic']=np.array(flat_list)
sarcasm_data

,text,sarcastic,processed_text,predictive_toxic
0,I know. I'm a Maker. You forgot the,1,know maker forget,0
1,Police praise A-League fans for overall good b...,1,police praise league fan overall good behaviou...,0
2,To the tune of Agadoo? Waterloo?,0,tune agadoo waterloo,0
3,Not sure if you're a troll or just fucking stu...,0,sure troll fucking stupid,1
4,The political ignorance is strong here on Redd...,1,political ignorance strong reddit know tpp lit...,1
...,...,...,...,...
204429,"On a similar note, I'm so glad Grawp's storyli...",0,similar note glad grawp storyline significantl...,0
204430,Cheeky Nando's with my mates after a piss up w...,0,cheeky nando mat piss bird lads go mental,1
204431,carriage ways allow people to look for parking...,0,carriage way allow people look park traffic ci...,0
204432,"Damn, this gets reposted once each week at lea...",1,damn get reposted week least op clearly say ev...,0


In [ ]:
sarcasm_data.to_csv('/content/gdrive/MyDrive/NLP_Project/sarcasm/sarcasm_data_with_predicted_toxic.csv', index=False)